In [100]:
import Pkg
#Pkg.add("JuMP")
#Pkg.add("GLPK")
#Pkg.add("Gruobi")
#Pkg.add("DataFrames")
#Pkg.add("CSV")
#Pkg.add("PyCall")

In [101]:
using JuMP, GLPK
using DataFrames
using CSV

In [102]:
# Load the data
scenarios_200 = CSV.read("../data/scenarios_200.csv", DataFrame)

# create a dictonary with 200 dataframes for each scenario
scenarios = Dict()
for i in 1:250
    df_helper = DataFrame(scenarios_200[:,3*i-2:3*i])
    #set column names
    rename!(df_helper, [:"price", :"wind power", :"grid_excess"])
    scenarios[i] = df_helper
end

In [114]:
scenarios[3]

Row,price,wind power,grid_excess
,Float64,Float64,Int64
1,119.7,21.3596,1
2,110.79,17.6276,0
3,104.84,15.5202,1
4,103.93,15.6879,1
5,103.77,13.6841,0
6,107.03,6.5381,1
7,127.17,3.91215,1
8,137.67,6.31816,1
9,144.57,11.0469,0


In [104]:
#scenarios[2]

In [105]:
# Initialize the DataFrame directly without dynamic column names
result_df = DataFrame(hour = Int[], p_DA = Float64[], delta = Float64[], delta_up = Float64[], delta_down = Float64[])

Row,hour,p_DA,delta,delta_up,delta_down
,Int64,Float64,Float64,Float64,Float64


In [106]:
W = 250
hours = 24

24

In [107]:
# Create a new model with GLPK solver
model = Model(GLPK.Optimizer)

# Define the decision variables for hour
@variable(model, p_DA[1:hours])
@variable(model, delta[1:W,1:hours])
@variable(model, delta_up[1:W,1:hours])
@variable(model, delta_down[1:W,1:hours])

# Define the objective function
@objective(model, Max, sum(1/W*(
        scenarios[i][hour,"price"] * p_DA[hour]
     + delta_up[i,hour] * scenarios[i][hour,"price"] * (scenarios[i][hour,"grid_excess"]*0.9 + (1-scenarios[i][hour,"grid_excess"])*1)
      - delta_down[i,hour] * scenarios[i][hour,"price"] * (scenarios[i][hour,"grid_excess"]*1 + (1-scenarios[i][hour,"grid_excess"])*1.2)
      ) for i in 1:W, hour in 1:hours))

# Define the constraints
@constraint(model, [hour in 1:hours], p_DA[hour] <= 200)
@constraint(model, [hour in 1:hours], p_DA[hour] >= 0)
@constraint(model, [i in 1:W, hour in 1:hours], delta[i,hour] == scenarios[i][hour,"wind power"] - p_DA[hour])
@constraint(model, [i in 1:W, hour in 1:hours], delta[i,hour] == delta_up[i,hour] - delta_down[i,hour])
@constraint(model, [i in 1:W, hour in 1:hours], delta_down[i,hour] >= 0)
#@constraint(model, [i in 1:W, hour in 1:hours], delta_down[i,hour] <= p_DA[hour])
@constraint(model, [i in 1:W, hour in 1:hours], delta_up[i,hour] >= 0)
#@constraint(model, [i in 1:W, hour in 1:hours], delta_up[i,hour] + p_DA[hour] <= 200)



# Solve the optimization problem
optimize!(model)


#println(model)

# Print the termination status
status = termination_status(model)
if status == MOI.OPTIMAL
    println("Optimal solution found")
    
    # RETURN OBJECTIVE value
    println("Objective value: ", objective_value(model))
else
    println("No optimal solution found")
end

In [108]:
println("Objective value: ", objective_value(model))
println("p_DA: ", value.(p_DA))
println("delta: ", value.(delta))
println("delta_up: ", value.(delta_up))
println("delta_down: ", value.(delta_down))

In [109]:
#make a table with the results
scenario = 1
for hour in 1:hours
    push!(result_df, [hour, value(p_DA[hour]), value(delta[scenario,hour]), value(delta_up[scenario,hour]), value(delta_down[scenario,hour])])
end

result_df

Row,hour,p_DA,delta,delta_up,delta_down
,Int64,Float64,Float64,Float64,Float64
1,1,51.5466,100.446,100.446,0.0
2,2,14.3897,129.07,129.07,0.0
3,3,51.149,91.4377,91.4377,0.0
4,4,45.9903,100.628,100.628,0.0
5,5,43.0742,107.062,107.062,0.0
6,6,43.3419,107.76,107.76,0.0
7,7,145.927,0.0,0.0,0.0
8,8,13.3411,129.92,129.92,0.0
9,9,1.1039,141.175,141.175,0.0
